# DEVNET-2449 Python For the Enterprise Automation Exloration

## Jupyter Notebooks
The *Jupyter Notebooks* in this workbench class are interactive learning environments. We are using them to learn about network device programability in combination with virtual Cisco network devices.

Notebooks are **interactive**, the code and text in each notebook can be changed on the fly and the result of those changes can be seen immediately within the notebook itself.

Notebooks are organized in *cells*, each displayed *rectangle* or *cell* is basically either Markdown text (like this cell) or Python code. Python code can be *executed* by either clicking the  <span class="inline">![](images/forwardicon.png)</span> in the 'Cell' menu in the menu bar or the key combination 'Shift-Enter'.

<div class="my-notify-warning">

<p>**IMPORTANT for Execution. PLEASE READ**. At its simplest, this is a playbook and execution is top-down. Code cells will look something like `In [#]:` (with '#' being a number). That's where you want to single-click. To execute the code, simply hit Shift+Enter.</p>

<p>The code should execute, and move onto the next cell. The number in `In [#]:` will change and if there is some return value (as opposed to an explicit `print()` statement) it will appear in an `Out [#]:` cell. The numbers will change after each cell execution.</p>

<p>It's easy to "execute" an entire notebook, task-by-task, by continuing to execute Shift+Return. Also, if you ever see `In [*]:` in your code cells (note the asterisk '\*'), it just means things are executing, and you are probably waiting on data to come back.</p>

<p>So depending on what you're doing, some patience might be required.</p>

</div>


## Environment
The workbench environment is running entirely inside of the provided MacBook Pros. We use [Vagrant](https://www.vagrantup.com/) to run two [CSR 1000V virtual routers](http://www.cisco.com/c/en/us/products/routers/cloud-services-router-1000v-series/index.html) running IOS XE 16.3.2 and we use [Docker](https://www.docker.com/) to provide the [Jupyter](http://jupyter.org/) interactive notebooks (like the document you are currently looking at).

The Python environment in the notebooks communicate with the routers via a host-only *internal* network on the MacBook Pro. The MacBook has its own IP address on this host-only network: `172.20.20.1`. The container can access the routers on their own IP addresses, `172.20.20.10` and `172.20.20.20` respectively. The following diagram shows a high level overview of the environment:

![Topology](images/topology.png)

We can access the CLI of the CSR1000v using a terminal (`Terminal.app`) and, while in the Vagrant directory, typing `vagrant ssh rtr1` or `vagrant ssh rtr2`. Because a public key has been installed into the router(s), we get a privileged shell / CLI without being prompted for a password.

While on the router, you can check the 'wsma' configuration section and also the remaining configuration which enables NETCONF and RESTCONF. 

Apart from a few static routes and an OSPF process there is not a lot else configured on the box.

Here we will set variables that we will use thoughout the session

In [4]:
HOST = '172.20.20.10'
PORT = 830
USER = 'vagrant'
PASS = 'vagrant'

Programmability has been around for a long time over the CLI API.  We all use it for FDN, but there are capabilities added over the years to communicate with devices via Python over ssh leveraging some form of python expect.  In this case we will use a tool called Netmiko.  However the format of the data is meant for human consumption, not a machine interface

In [5]:
import jtextfsm as textfsm
from netmiko import ConnectHandler
import pprint

def get_show_inventory(ip, username, password, enable_secret):
    # establish a connection to the device
    ssh_connection = ConnectHandler(
        device_type='cisco_ios',
        ip=ip,
        username=username,
        password=password,
        secret=enable_secret)
    # enter enable mode
    ssh_connection.enable()
    # prepend the command prompt to the result (used to identify the local device)
    result = ssh_connection.find_prompt() + "\n"
    # execute the show inventory command with extended delay
    result += ssh_connection.send_command("show inventory", delay_factor=2)
    # close SSH connection
    ssh_connection.disconnect()
    return result


If we are lucky there is some clear format of the response that we can leverage regular expression filters to turn the response into programming data types

In [8]:
    raw_text_data = get_show_inventory(HOST, USER, PASS, PASS)
    print ('This is what the text looks like\n' + raw_text_data)
    # Run the text through the FSM. 
    # The argument 'template' is a file handle and 'raw_text_data' is a 
    # string with the content from the show_inventory.txt file
    template = open("/home/docker/cli/show_inventory_multiple.textfsm")
    re_table = textfsm.TextFSM(template)
    fsm_results = re_table.ParseText(raw_text_data)
    # build keys for inventory sub-items 
    keys = [s for s in re_table.header]
    #build inventory list of dictionaries
    inventory = []
    for row in fsm_results:
      entry = {}
      for i, item in enumerate(row):
        entry.update({keys[i]:item})
      inventory.append(entry)
    pp = pprint.PrettyPrinter(indent=2)

This is what the text looks like
rtr1#
NAME: "Chassis", DESCR: "Cisco CSR1000V Chassis"
PID: CSR1000V          , VID: V00  , SN: 9H79916HPTZ

NAME: "module R0", DESCR: "Cisco CSR1000V Route Processor"
PID: CSR1000V          , VID: V00  , SN: JAB1303001C

NAME: "module F0", DESCR: "Cisco CSR1000V Embedded Services Processor"
PID: CSR1000V          , VID:      , SN:            




Here we can see what the template looks like

In [9]:
    print(template.read())

Value Filldown hostname (\S+)
Value name (.+)
Value description (.*)
Value productid (\S*)
Value vid (\S*)
Value Required serialnumber (\S+)

Start
  ^${hostname}[>#].*
  ^NAME: "${name}", DESCR: "${description}"
  ^PID: ${productid}.*VID: ${vid}.*SN: ${serialnumber} -> Record


Now that we have srtuctured data, lets do something interesting with it.  We can choose how we display it or to even search for a value of interest.

In [10]:
    print('Results translated through regular expression template organized as a dictonary')
    pp.pprint(inventory) # Print full inventory as dictionary
    print([key for key, value in inventory[0].items()])  # print key values as list
    for item in inventory:   # print inventory elements a table 
        print([item[key] for key in item])
    

Results translated through regular expression template organized as a dictonary
[ { u'description': u'Cisco CSR1000V Chassis',
    u'hostname': u'rtr1',
    u'name': u'Chassis',
    u'productid': u'CSR1000V',
    u'serialnumber': u'9H79916HPTZ',
    u'vid': u'V00'},
  { u'description': u'Cisco CSR1000V Route Processor',
    u'hostname': u'rtr1',
    u'name': u'module R0',
    u'productid': u'CSR1000V',
    u'serialnumber': u'JAB1303001C',
    u'vid': u'V00'}]
[u'serialnumber', u'name', u'vid', u'description', u'hostname', u'productid']
[u'9H79916HPTZ', u'Chassis', u'V00', u'Cisco CSR1000V Chassis', u'rtr1', u'CSR1000V']
[u'JAB1303001C', u'module R0', u'V00', u'Cisco CSR1000V Route Processor', u'rtr1', u'CSR1000V']


In [12]:
    sought_product = 'devicex'
    resultlist = [d for d in inventory if d.get('productid', '') == sought_product]
    print('\nFound Devices with specific productid ' + sought_product)
    pp.pprint(resultlist) #print list of devices with sought productid
    #print ([(item['hostname'], item['productid']) for item in resultlist])  #print found items as a tuple


Found Devices with specific productid devicex
[]


However, we are not always so lucky to have data that we can easily parse.

In [13]:
def get_show_ip_route(ip, username, password, enable_secret):
    # establish a connection to the device
    ssh_connection = ConnectHandler(
        device_type='cisco_ios',
        ip=ip,
        username=username,
        password=password,
        secret=enable_secret)
    # enter enable mode
    ssh_connection.enable()
    # prepend the command prompt to the result (used to identify the local device)
    result = ssh_connection.find_prompt() + "\n"
    # execute the show inventory command with extended delay
    result += ssh_connection.send_command("show ip route", delay_factor=2)
    # close SSH connection
    ssh_connection.disconnect()
    return result

In [14]:
raw_text_data = get_show_ip_route(HOST, USER, PASS, PASS)
print ('This is what the text looks like\n' + raw_text_data)

This is what the text looks like
rtr1#
Codes: L - local, C - connected, S - static, R - RIP, M - mobile, B - BGP
       D - EIGRP, EX - EIGRP external, O - OSPF, IA - OSPF inter area 
       N1 - OSPF NSSA external type 1, N2 - OSPF NSSA external type 2
       E1 - OSPF external type 1, E2 - OSPF external type 2
       i - IS-IS, su - IS-IS summary, L1 - IS-IS level-1, L2 - IS-IS level-2
       ia - IS-IS inter area, * - candidate default, U - per-user static route
       o - ODR, P - periodic downloaded static route, H - NHRP, l - LISP
       a - application route
       + - replicated route, % - next hop override, p - overrides from PfR

Gateway of last resort is 10.0.2.2 to network 0.0.0.0

S*    0.0.0.0/0 [254/0] via 10.0.2.2
      10.0.0.0/8 is variably subnetted, 2 subnets, 2 masks
C        10.0.2.0/24 is directly connected, GigabitEthernet1
L        10.0.2.15/32 is directly connected, GigabitEthernet1
      172.16.0.0/16 is variably subnetted, 2 subnets, 2 masks
C        172.16.

So let's take a look at an aternative method of interacting with devices.  Let's consider the machine's needs and leverage a model built for it.  That brings us to NETCONF.  Let's try to grab the routing table via programmatic API.

In [15]:
import ncclient.manager
import xml.dom.minidom
from ncclient.operations import TimeoutExpiredError
  
route_filter = '''
  <routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
    <routing-instance>
      <ribs>
        <rib>
          <name>ipv4-default</name>
          <address-family xmlns:v4ur="urn:ietf:params:xml:ns:yang:ietf-ipv4-unicast-routing">v4ur:ipv4-unicast</address-family>
          <default-rib>true</default-rib>
          <routes/>
        </rib>
      </ribs>
    </routing-instance>
  </routing-state>
  '''
nckwargs = dict(
    host=HOST,
    port=PORT,
    hostkey_verify=False,
    username=USER,
    password=PASS,
    device_params={'name':"csr"}
)
m = ncclient.manager.connect(**nckwargs)  
try:
    print ('Here we are printing the RIB as XML\n')
    c = m.get(filter=('subtree', route_filter))
    xmlDom = xml.dom.minidom.parseString(str(c))
    print (xmlDom.toprettyxml( indent = " " ))
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("ERORR severity={}, tag={}".format(e.severity, e.tag))

Here we are printing the RIB as XML

<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:9cebf572-d260-4bfc-818e-c26930db577b" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
   <routing-instance>
    <ribs>
     <rib>
      <name>ipv4-default</name>
      <address-family>v4ur:ipv4-unicast</address-family>
      <default-rib>true</default-rib>
      <routes>
       <route>
        <destination-prefix>0.0.0.0/0</destination-prefix>
        <route-preference>254</route-preference>
        <metric>0</metric>
        <next-hop>
         <next-hop-address>10.0.2.2</next-hop-address>
        </next-hop>
        <source-protocol>static</source-protocol>
       </route>
       <route>
        <destination-prefix>10.0.2.0/24</destination-prefix>
        <route-preference>0</route-preference>
        <metric>0</metric>
        <next-hop>
         <outgoing-interface>G

Now that is a lot of information to look through, maybe you are only interested in a few pieces of information per route.  We can apply an XPATH filter, which is to say we will parse the response programmatically and return the interesting data in a condensed format

In [17]:
ns = 'urn:ietf:params:xml:ns:yang:ietf-routing'
nsmap = dict(rt=ns)
route_table=[]
routes = c.data.findall(".//{%s}route" % ns)
for i, route in enumerate(routes):
    dest = route.xpath('./rt:destination-prefix/text()', namespaces=nsmap)
    hop = route.xpath('./rt:next-hop/rt:next-hop-address/text()', namespaces=nsmap)
    intfc = route.xpath('./rt:next-hop/rt:outgoing-interface/text()', namespaces=nsmap)
    prot = route.xpath('./rt:source-protocol/text()', namespaces=nsmap)
    if len(hop) > 0:
        entry = {'hop':hop[0]}
    if len(intfc) > 0:
        entry = {'intfc':intfc[0]}  
    entry.update({'dest':dest[0], 'prot':prot[0]}) 
    route_table.append(entry)
print ('\nOr we can just grab the information that is interesting\n')
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(route_table)  


Or we can just grab the information that is interesting

[ { 'dest': '0.0.0.0/0', 'hop': '10.0.2.2', 'prot': 'static'},
  { 'dest': '10.0.2.0/24', 'intfc': 'GigabitEthernet1', 'prot': 'direct'},
  { 'dest': '10.0.2.15/32', 'intfc': 'GigabitEthernet1', 'prot': 'direct'},
  { 'dest': '172.16.100.0/24', 'intfc': 'GigabitEthernet3', 'prot': 'direct'},
  { 'dest': '172.16.100.10/32', 'intfc': 'GigabitEthernet3', 'prot': 'direct'},
  { 'dest': '172.20.20.0/24', 'intfc': 'GigabitEthernet2', 'prot': 'direct'},
  { 'dest': '172.20.20.10/32', 'intfc': 'GigabitEthernet2', 'prot': 'direct'},
  { 'dest': '192.168.0.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.1.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.2.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.4.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '198.18.10.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '198.18.11.0/24', 'hop': '172.20.20

That seems easier to work with, but what are the items we can request?  In this case we can look at the Model.
There are to types of models, native (complete capability from the vendor) or the common models like ietf or openconfig.

In [19]:
from subprocess import Popen, PIPE, STDOUT
SCHEMA_TO_GET = 'ietf-interfaces'
print(' We are connecting to the device and downloading the YANG Module')
print("Now we will do a 'pyang -f tree of %s.yang'" % SCHEMA_TO_GET)
print('This structure is what data we can get (ro/rw) or set (rw)')
c = m.get_schema(SCHEMA_TO_GET)
p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
stdout_data = p.communicate(input=c.data)[0]
print stdout_data

 We are connecting to the device and downloading the YANG Module
Now we will do a 'pyang -f tree of ietf-interfaces.yang'
This structure is what data we can get (ro/rw) or set (rw)
module: ietf-interfaces
   +--rw interfaces
   |  +--rw interface* [name]
   |     +--rw name                        string
   |     +--rw description?                string
   |     +--rw type                        identityref
   |     +--rw enabled?                    boolean
   |     +--rw link-up-down-trap-enable?   enumeration {if-mib}?
   +--ro interfaces-state
      +--ro interface* [name]
         +--ro name               string
         +--ro type               identityref
         +--ro admin-status       enumeration {if-mib}?
         +--ro oper-status        enumeration
         +--ro last-change?       yang:date-and-time
         +--ro if-index           int32 {if-mib}?
         +--ro phys-address?      yang:phys-address
         +--ro higher-layer-if*   interface-state-ref
         +--ro lower

Now that we know whats available, we can grab operational data for the device to give us some perspective of its state.  In the coming months this will instead be a telemetry stream that a client could register for vs a poll.  However, we can see the advantage of how we can manipulate the information

In [21]:
cpu_util = '''<filter>
<cpu-usage xmlns="urn:cisco:params:xml:ns:yang:cisco-process-cpu">
<cpu-utilization>
</cpu-utilization>
</cpu-usage></filter>'''

cpu_process = '''<filter>
<cpu-usage xmlns="urn:cisco:params:xml:ns:yang:cisco-process-cpu" xmlns:y="http://tail-f.com/ns/rest" xmlns:pcpu="urn:cisco:params:xml:ns:yang:cisco-process-cpu">
<process-cpu-usage>
</process>
</process-cpu-usage>
</filter>'''
print ('Using NETCONF, you can present data as XML without any conversion')
cpu_util = str( m.get( filter=cpu_util ) )
xmlDom = xml.dom.minidom.parseString(cpu_util)
print xmlDom.toprettyxml( indent = " " )

Using NETCONF, you can present data as XML without any conversion
<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:932082be-6b01-496c-bfb9-68b6dd29368a" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <cpu-usage xmlns="urn:cisco:params:xml:ns:yang:cisco-process-cpu">
   <cpu-utilization>
    <five-seconds>0</five-seconds>
    <five-seconds-intr>0</five-seconds-intr>
    <one-minute>0</one-minute>
    <five-minutes>0</five-minutes>
   </cpu-utilization>
  </cpu-usage>
 </data>
</rpc-reply>



In [23]:
import xmltodict
print ('Or you can convert to Dictionary')
cpu_dict = xmltodict.parse( cpu_util )['rpc-reply']['data']
print [(i,v) for i,v in cpu_dict['cpu-usage']['cpu-utilization'].iteritems()]

Or you can convert to Dictionary
[(u'five-seconds', u'0'), (u'five-seconds-intr', u'0'), (u'one-minute', u'0'), (u'five-minutes', u'0')]


With structured data you can select data thats interesting

In [25]:
print ('\nWith structured data you can filter or sort \nHere we are grabbing the processes with higher run-time')
cpu_dict = xmltodict.parse( str( m.get( filter=cpu_process ) ) )
cpu_dict = cpu_dict['rpc-reply']['data']
nonzero_process = [t for t in cpu_dict['cpu-usage']['process-cpu-usage']['process'] if int(t['total-run-time']) > 100]
for i in nonzero_process: print (i['name'],i['pid'],i['total-run-time'])


With structured data you can filter or sort 
Here we are grabbing the processes with higher run-time
(u'Load Meter', u'2', u'457')
(u'MCP TIPC', u'3', u'392')
(u'Check heaps', u'9', u'10706')
(u'IPC Mcast Pendin', u'23', u'115')
(u'GraphIt', u'53', u'122')
(u'crypto sw pk pro', u'61', u'1046')
(u'Net Background', u'69', u'641')
(u'IOSD ipc task', u'76', u'2798')
(u'PuntInject Keepa', u'83', u'416')
(u'Environmental Mo', u'90', u'115')
(u'DBAL EVENTS', u'93', u'360')
(u'REDUNDANCY peer', u'95', u'118')
(u'100ms check', u'96', u'1558')
(u'IOSXE-RP Punt Se', u'118', u'252')
(u'Per-minute Jobs', u'129', u'2284')
(u'Per-Second Jobs', u'130', u'237')
(u'ONEP Application', u'131', u'1510')
(u'FPDB Timer Proce', u'134', u'210')
(u'fanrp_l2fib', u'135', u'140')
(u'DiagCard1/-1', u'178', u'470')
(u'FHRP Main thread', u'183', u'122')
(u'VRRS Main thread', u'187', u'1999')
(u'IPAM Manager', u'196', u'3107')
(u'IP ARP Retry Age', u'199', u'2988')
(u'ONEP CLI Command', u'213', u'13694')
(u'SSS Feat

Let's take a look at the interface model and take a look at operational data

In [30]:
CONF_FILTER ='''<filter>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface></interface>
  </interfaces>
</filter>'''
STATE_FILTER ='''<filter>
  <interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface><statistics><in-octets></in-octets></statistics></interface>
  </interfaces-state>
</filter>'''
print('\nHere is the XML data for Interfaces Operational State')
interfaces = m.get(STATE_FILTER)
pp = pprint.PrettyPrinter(indent=2)
print(xml.dom.minidom.parseString(interfaces.xml).toprettyxml(indent=" "))


Here is the XML data for Interfaces Operational State
<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:01868d05-1608-4c64-8089-51398f05af3c" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
   <interface>
    <statistics>
     <in-octets>392250</in-octets>
    </statistics>
   </interface>
   <interface>
    <statistics>
     <in-octets>1322447</in-octets>
    </statistics>
   </interface>
   <interface>
    <statistics>
     <in-octets>0</in-octets>
    </statistics>
   </interface>
   <interface>
    <statistics>
     <in-octets>0</in-octets>
    </statistics>
   </interface>
   <interface>
    <statistics>
     <in-octets>0</in-octets>
    </statistics>
   </interface>
  </interfaces-state>
 </data>
</rpc-reply>



and configuration data

In [28]:
m = ncclient.manager.connect(**nckwargs)
print('\nHere is the XML data for Interfaces Configuration')
interfaces = m.get_config('running',CONF_FILTER)
print(xml.dom.minidom.parseString(interfaces.xml).toprettyxml(indent=" "))


Here is the XML data for Interfaces Configuration
<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:4e8947a0-18ff-482e-a090-24a20b01a310" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
   <interface>
    <name>GigabitEthernet1</name>
    <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
    <enabled>true</enabled>
    <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
    <ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
   </interface>
   <interface>
    <name>GigabitEthernet2</name>
    <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
    <enabled>true</enabled>
    <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
     <address>
      <ip>172.20.20.10</ip>
      <netmask>255.255.255.0</netmask>
     </address>
    </ipv4>
    <ipv6 xmlns="urn:ietf:params:xml:ns:

Now that we are all Python experts, let's take a different perspective.  Models are similar to Programming languages.  Containers are to Classes, Lists are to Lists, Leafs are to Objects or types.

In a community effort we have "compiled" the YANG model in to Python code.  By importing a few libraries we can affect change on networking elements simply by manipulating code.

In [32]:
from ydk.services import CRUDService
from ydk.providers import NetconfServiceProvider
from ydk.models.ietf import ietf_interfaces as intf
from ydk.models.ietf import iana_if_type as iftype

# create NETCONF provider
provider = NetconfServiceProvider(address=HOST,
                                  port=PORT,
                                  username=USER,
                                  password=PASS)

# create CRUD service
crud = CRUDService()

# query object
q_i = intf.Interfaces()

# get stuff
intfs = crud.read(provider, q_i)

# print interface names and types
for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))

GigabitEthernet1, ethernetCsmacd, None
GigabitEthernet2, ethernetCsmacd, None
GigabitEthernet3, ethernetCsmacd, None
Loopback999, softwareLoopback, None
Tunnel4, tunnel, None


In [33]:
q_i = intf.InterfacesState()

# get stuff
intfs = crud.read(provider, q_i)
q_i = intf.InterfacesState()

# get stuff
intfs = crud.read(provider, q_i)
int_info=[(i.name, i.statistics.out_pkts, int(i.speed)/1000000, i.oper_status) for i in intfs.interface]
for thing in int_info: print(str(thing))

('GigabitEthernet1', 1916, 1000, <OperStatusEnum.up: 1>)
('GigabitEthernet2', 5265, 1000, <OperStatusEnum.up: 1>)
('GigabitEthernet3', 7728, 10000, <OperStatusEnum.up: 1>)
('Loopback999', 0, 8000, <OperStatusEnum.up: 1>)
('Tunnel4', 0, 0, <OperStatusEnum.down: 2>)


# Create Software Loopback

In [37]:
# new interface
new_loopback = intf.Interfaces.Interface()

# create a new loopback
new_loopback.name = 'Loopback666'
new_loopback.type = iftype.SoftwareloopbackIdentity()
new_loopback.description = 'Created by DevNet2449'
res = crud.create(provider, new_loopback)

In [38]:
# get stuff
q_i = intf.Interfaces()
intfs = crud.read(provider, q_i)

for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))

GigabitEthernet1, ethernetCsmacd, None
GigabitEthernet2, ethernetCsmacd, None
GigabitEthernet3, ethernetCsmacd, None
Loopback666, softwareLoopback, Created by DevNet2449
Loopback999, softwareLoopback, None
Tunnel4, tunnel, None


# Delete Software Loopback

In [39]:
# create CRUD service
crud = CRUDService()

# interface to delete
to_del = intf.Interfaces.Interface()

# create a new loopback
to_del.name = 'Loopback666'
res = crud.delete(provider, to_del)

In [40]:
# get stuff
intfs = crud.read(provider, q_i)

for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))
provider.close()

GigabitEthernet1, ethernetCsmacd, None
GigabitEthernet2, ethernetCsmacd, None
GigabitEthernet3, ethernetCsmacd, None
Loopback999, softwareLoopback, None
Tunnel4, tunnel, None
